In [ ]:
#  ФУНКЦИЯ ИЩЕТ ССЫЛКИ, ДАТУ, НАЗВАНИЕ СТАТЬИ, количество лайков НА ОДНОЙ СТРАНИЦЕ
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 10)  # выбор количество просмотра столбцов
pd.set_option('display.width', 9999)  # просмотр всех строк датафрейма, предотвращает перенос строк


def search_queries(query):
        url = 'https://habr.com/ru/search'
        search_parameters = {'q': query, 'target_type': 'posts', 'order': 'relevance'}
        queries_soup = BeautifulSoup(requests.get(url, search_parameters).text)

        result_of_research = queries_soup.find_all('article', 'tm-articles-list__item')  # можно: result_of_research = queries_soup.find_all('div', 'tm-article-snippet')
        list_query = []

        for x in result_of_research:
            link = x.find('a', 'tm-title__link').get('href')  # можно: link = x.find('h2').find_next('a').get('href')
            data = x.find('a').find_next('time').get('title')  # можно: data = x.find('a').find_next('time').text
            heading = x.find('a', 'tm-title__link').find_next('span').text
            like = x.find('div','tm-votes-meter tm-data-icons__item').find_next('span').get('title')

            full_url = 'https://habr.com' + link

            list_query.append({'Дата': data, 'Заголовок': heading, 'Ссылка на материал': full_url, 'количество лайков': like})

        info_df = pd.DataFrame(list_query)
        return info_df


search_1 = search_queries('ипотека')

search_1

In [ ]:
## ПОИСК ВСЕХ СТРАНИЦ
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup


def search_queries(query, page):
    all_list_query = []
    url = 'https://habr.com/ru/search/page1'
    search_parameters = {'q': query, 'target_type': 'posts', 'order': 'relevance'}
    for i in range(1, page + 1):
        url = url[:-1] + str(i)
        queries_soup = BeautifulSoup(requests.get(url, search_parameters).text)
        time.sleep(0.2)

        result_of_research = queries_soup.find_all('article', 'tm-articles-list__item')  # можно: result_of_research = queries_soup.find_all('div', 'tm-article-snippet')
        list_query = []

        for x in result_of_research:
            link = x.find('a', 'tm-title__link').get('href')  # можно: link =x.find('h2').find_next('a').get('href')

            list_query.append(link)

        list_full = list(map(lambda url_: 'https://habr.com' + url_, list_query))

        all_list_query += list_full

    return all_list_query


print(len(search_queries('pandas', 4)))
search_queries('pandas', 4)

ДОРАБОТКА ФУНКЦИИ

In [223]:
%%time

my_search_list = ['Московский драматический театр', 'Процентная ставка ЦБ']
#my_search_list = ['python', 'анализ данных']
number_page = 2

import pandas as pd
import time
import requests
from bs4 import BeautifulSoup


#  ФУНКЦИЯ ФОРМИРУЕТ СПИСОК ИЗ URL_ССЫЛОК ВСЕХ СТРАНИЦ ЗАПРОСА
def search_queries(query, page):

    fullall_list_query = []

    for query in my_search_list:
      url = 'https://habr.com/ru/search/page1'
      search_parameters = {'q': query, 'target_type': 'posts', 'order': 'relevance'}
      for i in range(1, page + 1):
          url = url[:-1] + str(i)
          queries_soup = BeautifulSoup(requests.get(url, search_parameters).text)
          time.sleep(0.2)

          result_of_research = queries_soup.find_all('article', 'tm-articles-list__item')  # можно: result_of_research = queries_soup.find_all('div', 'tm-article-snippet')

          list_query = []

          for x in result_of_research:
              link = x.find('a', 'tm-title__link').get('href')  # можно: link =x.find('h2').find_next('a').get('href')

              list_query.append(link)

          list_full = list(map(lambda url_: 'https://habr.com' + url_, list_query))

          fullall_list_query += list_full

    return fullall_list_query

list_url_search_1 = search_queries(my_search_list, number_page) # полностью весь лист с ссылками по всем из списка запросов + по всем страница!!!

pd_url_search_1 = pd.DataFrame({'ССЫЛКА': list_url_search_1}) # датафрейм по всем запросам + по всем страницам!!!



CPU times: user 599 ms, sys: 5.54 ms, total: 604 ms
Wall time: 9.9 s


In [219]:
pd_url_search_1

,ССЫЛКА
0,https://habr.com/ru/articles/821007/
1,https://habr.com/ru/companies/otus/articles/81...
2,https://habr.com/ru/news/750782/
3,https://habr.com/ru/news/752190/
4,https://habr.com/ru/news/747966/
...,...
75,https://habr.com/ru/companies/dit/articles/256...
76,https://habr.com/ru/articles/223903/
77,https://habr.com/ru/companies/JetBrains/articl...
78,https://habr.com/ru/articles/290882/


In [225]:
# ИНФОРМАЦИОННАЯ ФУНКЦИЯ  url_ссылок
list_url_search_1

def info_links(links_list):
  row_list = []
  for link in links_list:
    link_soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.1)

    data = link_soup.find('span').find_next('time').get('title')

    heading = link_soup.find('h1').find_next('span').text

    text_info_link = link_soup.find_all(xmlns='http://www.w3.org/1999/xhtml')  # в основном текст находится в тегах 'p', но на сайте не мало и в теге 'div'
    full_info_text = '\n'.join(map(lambda txt: txt.text ,text_info_link))

    #like = link_soup.find(class_='tm-votes-lever tm-votes-lever tm-votes-lever_appearance-article tm-article-rating__votes-switcher')
    #like = link_soup.find(class_='tm-svg-img tm-votes-meter__icon tm-votes-meter__icon tm-votes-meter__icon_appearance-article').find('title').text

    # ПАДАЕТ НА 32 индексе т.к там нет совсем  тега title и других тегов с лайками ( https://habr.com/ru/companies/smartengines/articles/820587/ )
    like = link_soup.find('svg').find_next('title').find_next('title') # ХАБР
    #like = link_soup.find('svg').find_next('title').text # MENU ????????

    row_list.append({'ДАТА': data, 'ЗАГОЛОВОК': heading, 'ТЕКСТ МАТЕРИАЛА': full_info_text, 'ЛАЙКИ': like})



  return pd.DataFrame(row_list) # формирую сразу информационный датафрейм

df_info_search_1 = info_links(list_url_search_1)

In [226]:
pd_full_search_1 = df_info_search_1.join(pd_url_search_1)

pd_full_search_1.info()
pd_full_search_1.head(8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ДАТА             80 non-null     object
 1   ЗАГОЛОВОК        80 non-null     object
 2   ТЕКСТ МАТЕРИАЛА  80 non-null     object
 3   ЛАЙКИ            80 non-null     object
 4   ССЫЛКА           80 non-null     object
dtypes: object(5)
memory usage: 3.2+ KB


,ДАТА,ЗАГОЛОВОК,ТЕКСТ МАТЕРИАЛА,ЛАЙКИ,ССЫЛКА
0,"2020-06-14, 14:35",Большой драматический театр 14 июня покажет пе...,\n\r\nСегодня в 21:00 мкс Большой драматически...,[Хабр],https://habr.com/ru/news/506622/
1,"2020-07-06, 13:10",Реальный спектакль в виртуальной среде: Большо...,\r\n14 июня Большой драматический театр им. Г....,[Хабр],https://habr.com/ru/companies/selectel/article...
2,"2017-11-08, 10:24",Театр и ИТ: Шекспиру и не снилось,Привычно читать о проникновении информационных...,[Хабр],https://habr.com/ru/companies/lanit/articles/3...
3,"2024-04-16, 21:10",1C Game Studios и Московский драматический теа...,Игровая студия 1C Game Studios и Московский др...,[Хабр],https://habr.com/ru/news/808215/
4,"2021-06-29, 18:11",Специалисты нашли уязвимость в системе доступа...,Мошенники публикуют в интернете схемы обхода т...,[Хабр],https://habr.com/ru/news/565318/
5,"2011-12-01, 18:52",Когда вы последний раз были в театре?,"\n\r\nСкорее всего, вам просто на просто никак...",[Хабр],https://habr.com/ru/companies/qiwi/articles/13...
6,"2009-01-22, 16:37",Интернет-театр,Компания Classic TV вскоре запустит новый прое...,[Хабр],https://habr.com/ru/articles/49893/
7,"2019-11-08, 19:42",В Театре на Таганке появится ИИ с голосом Влад...,\n\r\nМосковский театр на Таганке анонсировал ...,[Хабр],https://habr.com/ru/news/475102/
